# Exploring the Datasets
*Authors: Angelika Shastapalava, Excel Espina, David Hadaller, Sam Mundle*  

### What are we using:  
1) The "Discovery" API is MTA's official developer resource to get real-time data from their NYC Bus Time service. You can get more information <a href="http://bustime.mta.info/wiki/Developers/Index">here</a>  
2) Kaggle's NYC Bus Data <a href="https://www.kaggle.com/stoney71/new-york-city-transport-statistics">here</a>

### How are we using it:
Using regression and classification techniques learned in class, we want to explore the following:  
> 1. Based on a ~10 stops/lines how closely do the actual stop times reflect the posted bus
schedules and what is the distribution around the scheduled time that busses actually
arrive?
>2. What environmental factors impact a buses schedule? What impact does time of day,
temperature, and weather have?
>3. What socioeconomic factors play into a buses schedule? Do we see better or worse
availability in neighborhoods with different average incomes?
>4. Predicting with a defined degree certainty if a bus is coming within a given time frame

### Sections:
1) [Loading the Datasets](#Loading-the-Datasets)  
2) [Cleaning the Data](#Cleaning-the-Data)  
3) [Visualizing the Data](#Visualizing-the-Data)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

### Loading the Datasets 
We want to work with the Kaggle dataset so head over <a href="https://www.kaggle.com/stoney71/new-york-city-transport-statistics">here</a> and download the zip file. (A word of caution: the dataset is approx **5GB** when extracted!)

After you extract the data, we want to load a csv on our notebook.  

The `error_bad_lines=False` parameter fixes some formatting issues when we load in our dataset.

In [28]:
%%capture
mta = pd.read_csv('mta_1708.csv', error_bad_lines=False)
# mta.set_index('PublishedLineName', inplace=True)

In [29]:
mta.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,2017-08-01 00:01:03,0,Q32,W 32 ST/7 AV,40.749405,-73.991020,JACKSON HTS NORTHERN - 81 via ROOSVLT,40.755322,-73.886139,NYCT_7424,40.749403,-73.990841,W 32 ST/AV OF THE AMERICAS,< 1 stop away,220.0,2017-08-01 00:01:37,24:01:11
1,2017-08-01 00:00:52,0,B35,39 ST/1 AV,40.656456,-74.012245,BROWNSVILLE M GASTON BL via CHURCH,40.656345,-73.907188,NYCT_406,40.651330,-73.938960,CHURCH AV/E 42 ST,approaching,107.0,2017-08-01 00:02:00,23:56:12
2,2017-08-01 00:01:18,1,Q83,227 ST/113 DR,40.702263,-73.730339,JAMAICA HILLSIDE - 153 via LIBERTY,40.706795,-73.804100,NYCT_6449,40.706532,-73.804177,153 ST/HILLSIDE AV,at stop,25.0,2017-08-01 00:01:27,24:00:00
3,2017-08-01 00:01:05,0,M60-SBS,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:39:14
4,2017-08-01 00:01:05,0,M60-SBS,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:44:32


In [30]:
mta.dtypes

RecordedAtTime                object
DirectionRef                   int64
PublishedLineName             object
OriginName                    object
OriginLat                    float64
OriginLong                   float64
DestinationName               object
DestinationLat               float64
DestinationLong              float64
VehicleRef                    object
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
NextStopPointName             object
ArrivalProximityText          object
DistanceFromStop             float64
ExpectedArrivalTime           object
ScheduledArrivalTime          object
dtype: object

## Cleaning the Data
For this EDA, we're only going to be looking at the M100 bus going to 

First things first: convert ```ExpectedArrivalTime```'s Timestamp format to just the time itself.

Then we need to adjust the ```ScheduledArrivalTime``` to match python's time range of 0-23 hours. Here we pass ```errors='coerce'``` parameter to convert troublesome or out of range times to NaT (Not a Time).

In [33]:
mta = mta[mta['VehicleRef'] == 'NYCT_7424']

#tips = tips[tips['time'] == 'Dinner'].head(5)

df1 = mta[mta['DirectionRef'] == 0]
df2 = mta[mta['DirectionRef'] == 1]

#tips[['total_bill', 'tip', 'smoker', 'time']].head(5)
df1 = df1[['DestinationName','DirectionRef']].rename(columns={'DestinationName':'Dest1','DirectionRef':'Dir1'})
df2 = df2[['DestinationName','DirectionRef']].rename(columns={'DestinationName':'Dest2','DirectionRef':'Dir2'})

pd.concat([df1, df2], axis=1)


,Dest1,Dir1,Dest2,Dir2
0,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
788,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
1551,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
2267,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
2959,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
3580,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
4185,JACKSON HTS NORTHERN - 81 via ROOSVLT,0.0,NaN,NaN
23889,NaN,NaN,JAMAICA MERRICK BL via 14 AV via MAIN ST,1.0
25921,NaN,NaN,JAMAICA MERRICK BL via 14 AV via MAIN ST,1.0
28118,NaN,NaN,JAMAICA MERRICK BL via 14 AV via MAIN ST,1.0


In [12]:
mta.head(20)

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
225,2017-08-01 00:01:04,1,M100,BROADWAY/W 219 ST,40.871967,-73.913040,E HARLM 1 AV-125 ST via BDWY via AMSTRDM,40.802273,-73.931145,NYCT_8366,40.872595,-73.912298,9 av/218 st,< 1 stop away,211.0,2017-08-01 00:01:26,24:08:49
234,2017-08-01 00:00:53,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4368,40.810075,-73.950935,W 125 ST/FRED DOUGLASS BL,approaching,67.0,2017-08-01 00:01:27,23:55:18
346,2017-08-01 00:01:08,1,M100,BROADWAY/W 219 ST,40.871967,-73.913040,E HARLM 1 AV-125 ST via BDWY via AMSTRDM,40.802273,-73.931145,NYCT_8375,40.854328,-73.933650,BROADWAY/W 187 ST,approaching,51.0,NaN,23:55:20
548,2017-08-01 00:01:09,1,M100,BROADWAY/W 219 ST,40.871967,-73.913040,E HARLM 1 AV-125 ST via BDWY via AMSTRDM,40.802273,-73.931145,NYCT_8388,40.809137,-73.948722,W 125 ST/ADAM C POWELL BL,approaching,88.0,2017-08-01 00:01:29,23:55:52
762,2017-08-01 00:01:25,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4349,40.851938,-73.934836,BROADWAY/W 185 ST,approaching,143.0,2017-08-01 00:01:50,23:57:26
1005,2017-08-01 00:11:28,1,M100,BROADWAY/W 219 ST,40.871967,-73.913040,E HARLM 1 AV-125 ST via BDWY via AMSTRDM,40.802273,-73.931145,NYCT_8366,40.869532,-73.915142,10 AV/W 214 ST,approaching,76.0,2017-08-01 00:11:51,24:10:29
1014,2017-08-01 00:11:37,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4368,40.827159,-73.946196,AMSTERDAM AV/W 149 ST,approaching,123.0,2017-08-01 00:12:21,24:06:52
1121,2017-08-01 00:11:35,1,M100,BROADWAY/W 219 ST,40.871967,-73.913040,E HARLM 1 AV-125 ST via BDWY via AMSTRDM,40.802273,-73.931145,NYCT_8375,40.839757,-73.940335,W 166 ST/ST NICHOLAS AV,at stop,18.0,NaN,24:03:47
1523,2017-08-01 00:11:39,0,M100,1 AV/125 ST,40.801968,-73.931358,INWOOD 220 ST via AMSTERDAM via BWAY,40.871902,-73.913101,NYCT_4349,40.859447,-73.922601,10 AV/HARLEM RIVER DR,at stop,18.0,2017-08-01 00:11:52,24:08:48
1754,2017-08-01 00:21:06,1,M100,BROADWAY/W 219 ST,40.871967,-73.913040,E HARLM 1 AV-125 ST via BDWY via AMSTRDM,40.802273,-73.931145,NYCT_8366,40.861532,-73.930241,BROADWAY/BENNETT AV,< 1 stop away,200.0,2017-08-01 00:21:37,24:21:36


In [5]:
### Removing Date from the ExpectedArrivalTime

# Convert object to datetime64 
mta['expected_time'] = pd.to_datetime(mta['ExpectedArrivalTime']).dt.time
mta['scheduled_time'] = pd.to_datetime(mta['ScheduledArrivalTime'],format='%H:%M:%S', errors='coerce').dt.time
# mta['scheduled_time'] = pd.to_datetime(mta['scheduled_time'])
# temp = pd.to_datetime(mta['scheduled_time']).dt.time
# datetime.datetime.strptime().time()
mta.infer_objects().dtypes

AttributeError: 'DataFrame' object has no attribute 'infer_objects'

In [6]:
mta.head()

,RecordedAtTime,DirectionRef,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,expected_time,scheduled_time
PublishedLineName,,,,,,,,,,,,,,,,,,
Q32,2017-08-01 00:01:03,0,W 32 ST/7 AV,40.749405,-73.991020,JACKSON HTS NORTHERN - 81 via ROOSVLT,40.755322,-73.886139,NYCT_7424,40.749403,-73.990841,W 32 ST/AV OF THE AMERICAS,< 1 stop away,220.0,2017-08-01 00:01:37,24:01:11,00:01:37,NaT
B35,2017-08-01 00:00:52,0,39 ST/1 AV,40.656456,-74.012245,BROWNSVILLE M GASTON BL via CHURCH,40.656345,-73.907188,NYCT_406,40.651330,-73.938960,CHURCH AV/E 42 ST,approaching,107.0,2017-08-01 00:02:00,23:56:12,00:02:00,23:56:12
Q83,2017-08-01 00:01:18,1,227 ST/113 DR,40.702263,-73.730339,JAMAICA HILLSIDE - 153 via LIBERTY,40.706795,-73.804100,NYCT_6449,40.706532,-73.804177,153 ST/HILLSIDE AV,at stop,25.0,2017-08-01 00:01:27,24:00:00,00:01:27,NaT
M60-SBS,2017-08-01 00:01:05,0,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:39:14,00:06:47,23:39:14
M60-SBS,2017-08-01 00:01:05,0,BROADWAY/W 106 ST,40.801819,-73.967644,SELECT BUS SERVICE LA GUARDIA AIRPORT,40.768074,-73.862091,NYCT_5846,40.770403,-73.917687,HOYT AV/31 ST,4.1 miles away,6519.0,2017-08-01 00:06:47,23:44:32,00:06:47,23:44:32


Now let's drop observations whose ```DistanceFromStop``` is farther away than 30 ft.

In [7]:
mta = mta.loc[(mta['DistanceFromStop']<=30),]
print(mta.shape)

(1856274, 18)


Now let's drop missing ```ExpectedArrivalTime, expected_time & scheduled_time``` since we can't impute it at the moment.

In [8]:
mta = mta.loc[(mta['ExpectedArrivalTime'].notnull()),]
mta = mta.loc[(mta['expected_time'].notnull()),]
mta = mta.loc[(mta['scheduled_time'].notnull()),]
print(mta.shape)

(1315851, 18)


We'll create a new column that calculates the time difference between the expected and scheduled.

In [9]:
mta['time_diff'] = (pd.to_timedelta(mta['expected_time'].astype(str)) - 
                   pd.to_timedelta(mta['scheduled_time'].astype(str)))

In [10]:
mta['time_diff_mins'] = ((mta['time_diff'] / np.timedelta64(1, 'm')) + 1440).astype(int)
mta.dtypes

RecordedAtTime                        object
DirectionRef                           int64
OriginName                            object
OriginLat                            float64
OriginLong                           float64
DestinationName                       object
DestinationLat                       float64
DestinationLong                      float64
VehicleRef                            object
VehicleLocation.Latitude             float64
VehicleLocation.Longitude            float64
NextStopPointName                     object
ArrivalProximityText                  object
DistanceFromStop                     float64
ExpectedArrivalTime                   object
ScheduledArrivalTime                  object
expected_time                         object
scheduled_time                        object
time_diff                    timedelta64[ns]
time_diff_mins                         int32
dtype: object

## Visualizing the Data

Now we can visualize the time difference between expected and scheduled arrival times.

In [ ]:
# TODO

Looking at MTA API:

In [ ]:
# Key for Excel
mta_key = "51b681ab-bb14-4f29-9104-db15a7a41d41"